In [3]:
import autosklearn.classification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import sklearn.metrics

In [4]:
dataset = pd.read_csv('../Datasets/Echocardiogram/echocardiogram.csv')

In [5]:
dataset.drop(['name', 'group', 'aliveat1'], axis=1, inplace=True)

In [6]:
dataset.dropna(subset=['alive'], inplace=True)

In [7]:
dataset.isna().sum()

survival                 1
alive                    0
age                      5
pericardialeffusion      0
fractionalshortening     7
epss                    14
lvdd                    10
wallmotion-score         3
wallmotion-index         1
mult                     3
dtype: int64

In [8]:
continuous_features = dataset.drop(['pericardialeffusion', 'alive'], 1).columns

In [9]:
continuous_features

Index(['survival', 'age', 'fractionalshortening', 'epss', 'lvdd',
       'wallmotion-score', 'wallmotion-index', 'mult'],
      dtype='object')

In [10]:
features_with_outliers = ['wallmotion-score', 'wallmotion-index', 'mult']

In [11]:
for feature in continuous_features:
    if feature in features_with_outliers:
        dataset[feature].fillna(dataset[feature].median(), inplace=True)
    else:
        dataset[feature].fillna(dataset[feature].mean(), inplace=True)

In [12]:
X, y = dataset.drop('alive', 1), dataset['alive']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# AUTOML

In [26]:
automl = autosklearn.classification.AutoSklearnClassifier()
automl.fit(X_train, y_train)
y_hat = automl.predict(X_test)

In [27]:
sklearn.metrics.accuracy_score(y_test, y_hat)

0.8484848484848485

In [28]:
import pickle
with open('./models/echocardiogram_automl.pkl', 'wb') as f:
    pickle.dump(automl, f)

In [1]:
import pickle
with open('./models/echocardiogram_automl.pkl', 'rb') as f:
    automl = pickle.load(f)

len(automl.show_models())

28

# RANDOM FOREST

In [11]:
import sklearn.ensemble

model = sklearn.ensemble.RandomForestClassifier(n_estimators=100, n_jobs=5, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(n_jobs=5, random_state=42)

In [12]:
y_hat = model.predict(X_test)
sklearn.metrics.accuracy_score(y_test, y_hat)

0.9090909090909091

In [13]:
import pickle
with open('./models/echocardiogram_rf.pkl', 'wb') as f:
    pickle.dump(model, f)

In [13]:
import pickle
with open('./models/echocardiogram_rf.pkl', 'rb') as f:
    model = pickle.load(f)

# LIME

In [12]:
import lime
import lime.lime_tabular
import tqdm 

In [13]:
categorical_features = X_train.columns.drop(continuous_features).tolist()
explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values, feature_names=X_train.columns.tolist(), class_names=['Dead', 'Alive'], categorical_features=categorical_features, discretize_continuous=True)

# LIME Global

In [14]:
from lime import submodular_pick
import time

start_time = time.time()
exp1 = submodular_pick.SubmodularPick(explainer, X_test.values, model.predict_proba, sample_size=500, num_features=len(X_test.columns), num_exps_desired=5)
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
exp2 = submodular_pick.SubmodularPick(explainer, X_test.values, model.predict_proba, sample_size=500, num_features=len(X_test.columns), num_exps_desired=5)
print("--- %s seconds ---" % (time.time() - start_time))

/home/krish/anaconda3/envs/interpret2/lib/python3.7/site-packages/lime/submodular_pick.py:58: UserWarning: Requested sample size larger than
                              size of input data. Using all data
  size of input data. Using all data""")


--- 112.38904762268066 seconds ---


/home/krish/anaconda3/envs/interpret2/lib/python3.7/site-packages/lime/submodular_pick.py:58: UserWarning: Requested sample size larger than
                              size of input data. Using all data
  size of input data. Using all data""")


--- 105.21687602996826 seconds ---


In [15]:
def get_feature_imp(sp_obj):
    W_pick=pd.DataFrame([dict(this.as_list(this.available_labels()[0])) for this in sp_obj.sp_explanations]).fillna(0)
    W_pick['prediction'] = [this.available_labels()[0] for this in sp_obj.sp_explanations]
    W=pd.DataFrame([dict(this.as_list(this.available_labels()[0])) for this in sp_obj.explanations]).fillna(0)
    W['prediction'] = [this.available_labels()[0] for this in sp_obj.explanations]
    np.abs(W.drop("prediction", axis=1)).mean(axis=0).sort_values(ascending=False).head(25).sort_values(ascending=True)
    grped_coeff = W.groupby("prediction").mean()
    grped_coeff = grped_coeff.T
    return grped_coeff[0].values

In [16]:
feat_imp1 = get_feature_imp(exp1)
feat_imp2 = get_feature_imp(exp2)

In [17]:
feat_imp1

array([ 0.03403393,  0.02104885,  0.01992007,  0.01381603,  0.01329216,
       -0.00717206,  0.02101801, -0.00538454, -0.00497413, -0.02907108,
       -0.01006555, -0.01282667,  0.00552578, -0.0031357 , -0.00283924,
        0.0011673 ,  0.00081109,  0.11373321,  0.01373385,  0.00960948,
        0.01100406, -0.00161056, -0.00513487,  0.00029384, -0.01419836,
        0.0060796 ,  0.        , -0.0003662 , -0.0004152 ,  0.06511537,
        0.00334918, -0.00069433, -0.01156011, -0.00061595])

In [18]:
def global_identity(feat_imp1, feat_imp2):
    sum = 0
    for i in range(len(feat_imp1)):
        if(feat_imp1[i] == feat_imp2[i]):
            sum += 1
    return sum/len(feat_imp1)

In [19]:
i = global_identity(feat_imp1, feat_imp2)
i

0.0

In [20]:
def normal_fi(feat_imp):
    return np.abs(feat_imp) / np.sum(np.abs(feat_imp))

In [21]:
normal_feat_imp = normal_fi(feat_imp1 + 1e-9)

In [22]:
#Entropy Ratio
Ser = np.sum(normal_feat_imp*np.log(normal_feat_imp))/np.log(1/len(normal_feat_imp))

# Kullback-Leibler Divergence
Skl = np.sum(normal_feat_imp*np.log(normal_feat_imp/(1/len(normal_feat_imp))))

In [23]:
def calc_gini(pfi):
    sum = 0
    for i in range(len(pfi)):
        sum_curr = 0
        for j in range(len(pfi)):
            sum_curr += np.abs(pfi[i]-pfi[j])
        sum += sum_curr
    
    return sum/(2*len(pfi)**2)*(np.sum(pfi)/len(pfi))

In [24]:
Sg = calc_gini(normal_feat_imp)

In [25]:
Ser, Skl, Sg

(0.7851652819800737, 0.7575846689425121, 0.0005424322225279798)

In [26]:
def calc_alpha_fi(normal_pfi, alpha):
    j_inst = 0
    sum = 0
    for i in range(len(normal_pfi)-1, -1, -1):
        sum += normal_pfi[i]
        if sum<=alpha:
            j_inst = i
        else:
            break
    return 1- (j_inst/len(normal_pfi))

In [27]:
calc_alpha_fi(normal_feat_imp, 0.8)

0.8529411764705882

In [28]:
def get_feature_imp_all(sp_obj):
    W_pick=pd.DataFrame([dict(this.as_list(this.available_labels()[0])) for this in sp_obj.sp_explanations]).fillna(0)
    W_pick['prediction'] = [this.available_labels()[0] for this in sp_obj.sp_explanations]
    W=pd.DataFrame([dict(this.as_list(this.available_labels()[0])) for this in sp_obj.explanations]).fillna(0)
    W['prediction'] = [this.available_labels()[0] for this in sp_obj.explanations]
    np.abs(W.drop("prediction", axis=1)).mean(axis=0).sort_values(ascending=False).head(25).sort_values(ascending=True)
    grped_coeff = W.groupby("prediction").mean()
    grped_coeff = grped_coeff.T
    return grped_coeff

In [29]:
class1_feat_imp, class2_feat_imp = get_feature_imp_all(exp1)[0].values, get_feature_imp_all(exp1)[1].values
normal_class1_fi, normal_class2_fi = normal_fi(class1_feat_imp), normal_fi(class2_feat_imp)

In [30]:
np.linalg.norm(normal_class1_fi - normal_class2_fi, ord=2)

0.7226940245884969

# CIU

In [14]:
from ciu import determine_ciu
import tqdm
import metrics

In [15]:
def enc_exp(exp, feature_num):
    enc_exp = np.zeros((len(exp),feature_num))
    for i in range(len(exp)):
        for j in range(len(exp[i])):
            enc_exp[i][int(exp[i,j,0])] = exp[i,j,1]
    return enc_exp

In [16]:
feat_list = X_train.columns.tolist()

In [17]:
def exp_fn_blk(xtest):
    exp1 = []
    for i in tqdm.tqdm(range(len(xtest))):
        exp = determine_ciu(X_test.iloc[i:i+1], model.predict_proba, X_train.to_dict('list'), samples = 1000, prediction_index = 1)
        exp_list = [[feat_list.index(i), exp.ci[i]] for i in exp.ci]
        exp1.append(exp_list)
    return np.array(exp1)

In [18]:
exp1 = exp_fn_blk(X_test)
exp2 = exp_fn_blk(X_test)

100%|██████████| 33/33 [00:31<00:00,  1.06it/s]


In [19]:
np.save('./explanations/echocardiogram_ciu1.npy', exp1)
np.save('./explanations/echocardiogram_ciu2.npy', exp2)

In [20]:
i = metrics.calc_identity(exp1, exp2)
s = metrics.calc_separability(exp1)
enc1 = enc_exp(exp1, len(feat_list))
sb = metrics.calc_stability(enc1, y_test)

/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [21]:
i, s, sb

((6.0606060606060606, 31, 33), (0, 33, 1089, 0.0), (6.0, 33))

In [22]:
X_test_norm = metrics.normalize_test(X_train, X_test)
sim = metrics.calc_similarity(exp1, X_test_norm)

In [23]:
sim

0.2553918733279871

In [24]:
list_monotonicity = []
list_non_sensitivity = []
list_effective_complexity = []

for i in tqdm.tqdm(range(len(X_test))):
    atr = exp1[i]
    sorted_atr = [j for i,j in atr]
    sorted_feat = [i for i,j in atr]
    y = np.zeros(2, dtype=int)
    np.put(y, y_test.iloc[i], 1)
    example = metrics.FeatureAttribution(model, X_test.to_numpy()[i], y, sorted_atr)
    list_monotonicity.append(example.monotonicity())
    list_non_sensitivity.append(example.non_sensitivity())
    list_effective_complexity.append(example.effective_complexity(sorted_feat, 0.1))

100%|██████████| 33/33 [02:02<00:00,  3.70s/it]


In [25]:
print(np.mean(list_monotonicity))
print(np.mean(list_non_sensitivity))
print(np.mean(list_effective_complexity))

print(np.median(list_monotonicity))
print(np.median(list_non_sensitivity))
print(np.median(list_effective_complexity))

0.23926402448173148
0.42424242424242425
4.545454545454546
0.29130434782608694
0.0
6.0


In [26]:
metrics.calc_trust_score(model, X_test.to_numpy(), exp1, 3, X_train.columns.to_list())

100%|██████████| 33/33 [06:17<00:00, 11.44s/it]


0.6565656565656565

# RULEFIT

In [11]:
from skrules import SkopeRules
import metrics_rules
import time

In [12]:
clf = SkopeRules(max_depth_duplication=2,
                    n_estimators=512,
                    precision_min=0.3,
                    recall_min=0.1,
                    feature_names=X_train.columns.tolist())

In [13]:
start_time = time.time()
clf.fit(X_train, y_train)
print("--- %s seconds ---" % (time.time() - start_time))

--- 8.621065616607666 seconds ---


In [14]:
start_time = time.time()
top_rules1 = clf.score_top_rules(X_test)
top_rules2 = clf.score_top_rules(X_test)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.048004150390625 seconds ---


In [15]:
i = metrics_rules.calc_identity_rules(top_rules1, top_rules2)
print(i)

s = metrics_rules.calc_separability_rules(top_rules1)
print(s)

enc_rules = metrics_rules.exp_enc(clf, top_rules1)
sb = metrics_rules.calc_stability_rules(enc_rules, y_test)
print(sb)

(0.0, 33, 33)
(270, 33, 1089, 24.793388429752067)
(2.0, 33)


/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [16]:
X_test_norm = metrics_rules.normalize_test(X_train, X_test)
sim = metrics_rules.calc_similarity(enc_rules, X_test_norm)
print(sim)

0.9080987582996202


# RULEMATRIX

In [12]:
categorical_features = X_train.columns.tolist()
continuous_features = X_train.columns.drop(categorical_features).tolist()

In [13]:
import rulematrix
import time
import metrics_rules

In [14]:
is_continuous = [True if i in continuous_features else False for i in X_train.columns.tolist()]
is_categorical = [True if i in categorical_features else False for i in X_train.columns.tolist()]

In [15]:
surrogate = rulematrix.surrogate.rule_surrogate(
    model.predict,
    X_train,
    sampling_rate=4,
    is_continuous=is_continuous,
    is_categorical=is_categorical,
    seed=42
)

In [16]:
test_x = X_test.to_numpy()

In [17]:
def exp_fn_blk(xtest):
    exp1 = []
    for i in range(len(xtest)):
        queried_rules = np.arange(surrogate.student.n_rules)[surrogate.student.decision_path(test_x[i].reshape(1,-1)).reshape(-1)]
        exp1.append(queried_rules[-1])
    return np.array(exp1)
exp_fn_wrap = lambda x: np.array(exp_fn_blk(x))

In [18]:
start_time = time.time()
exp1 = exp_fn_blk(test_x)
exp2 = exp_fn_blk(test_x)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.012479782104492188 seconds ---


In [19]:
def enc_exp(exp, n_features):
    enc = []
    for i in range(exp.shape[0]):
        new = np.zeros(n_features)
        for j in surrogate.student.rule_list[exp[i]].clauses:
            new[j.feature_idx] = 1
        enc.append(new)
    return np.array(enc)

In [20]:
enc_exp = enc_exp(exp1, X_train.shape[1])

In [21]:
i = metrics_rules.calc_identity_rules(exp1, exp2)
print(i)

s = metrics_rules.calc_separability_rules(exp1)
print(s)

sb = metrics_rules.calc_stability_rules(enc_exp, y_test)
print(sb)

(0.0, 33, 33)
(680, 33, 1089, 62.44260789715335)
(4.0, 33)


/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [22]:
X_test_norm = metrics_rules.normalize_test(X_train, X_test)
sim = metrics_rules.calc_similarity(enc_exp, X_test_norm)

In [23]:
sim

0.3342516069788797

# ANCHOR Global


In [17]:
from anchor import anchor_tabular
import anchor_utils
import tqdm

In [18]:
explainer = anchor_tabular.AnchorTabularExplainer(
    y_train.unique().tolist(),
    X_train.columns.tolist(),
    X_train.values
)

In [19]:
# Feature Importance using Anchor
def calc_fi(X_test, model, explainer):
    all_exps = []
    for i in tqdm.tqdm(range(len(X_test))):
        exp = explainer.explain_instance(X_test.values[i], model.predict, threshold=0.95)
        all_exps.append(exp.exp_map)
    fi = anchor_utils.greedy_pick_anchor(all_exps, X_test.values, k = len(X_test.columns))
    return fi
        

In [20]:
exp1 = calc_fi(X_test, model, explainer)
exp2 = calc_fi(X_test, model, explainer)

100%|██████████| 33/33 [01:36<00:00,  2.91s/it]


0 0.15151515151515152
1 0.21212121212121213
2 0.2727272727272727
3 0.30303030303030304
4 0.3333333333333333
5 0.36363636363636365
6 0.3939393939393939
7 0.42424242424242425
8 0.45454545454545453


100%|██████████| 33/33 [01:12<00:00,  2.19s/it]

0 0.15151515151515152
1 0.21212121212121213
2 0.24242424242424243
3 0.2727272727272727
4 0.30303030303030304
5 0.3333333333333333
6 0.36363636363636365
7 0.3939393939393939
8 0.42424242424242425


In [21]:
def normal_fi(feat_imp):
    feat_imp = np.array(feat_imp) + 1e-9
    return np.abs(feat_imp) / np.sum(np.abs(feat_imp))

In [22]:
normal_feat_imp1 = normal_fi(exp1)
normal_feat_imp2 = normal_fi(exp2)

In [23]:
def global_identity(feat_imp1, feat_imp2):
    sum = 0
    for i in range(len(feat_imp1)):
        if(feat_imp1[i] == feat_imp2[i]):
            sum += 1
    return sum/len(feat_imp1)

i = global_identity(normal_feat_imp1, normal_feat_imp2)
i


0.0

In [24]:
#Entropy Ratio
Ser = np.sum(normal_feat_imp1*np.log(normal_feat_imp1))/np.log(1/len(normal_feat_imp1))

# Kullback-Leibler Divergence
Skl = np.sum(normal_feat_imp1*np.log(normal_feat_imp1/(1/len(normal_feat_imp1))))

In [25]:
def calc_gini(pfi):
    sum = 0
    for i in range(len(pfi)):
        sum_curr = 0
        for j in range(len(pfi)):
            sum_curr += np.abs(pfi[i]-pfi[j])
        sum += sum_curr
    
    return sum/(2*len(pfi)**2)*(np.sum(pfi)/len(pfi))

In [26]:
Sg = calc_gini(normal_feat_imp1)

In [27]:
Ser, Skl, Sg

(0.8819555297531368, 0.25937021124504167, 0.00445169138315067)

In [28]:
def calc_alpha_fi(normal_pfi, alpha):
    j_inst = 0
    sum = 0
    for i in range(len(normal_pfi)-1, -1, -1):
        sum += normal_pfi[i]
        if sum<=alpha:
            j_inst = i
        else:
            break
    return 1- (j_inst/len(normal_pfi))

In [29]:
calc_alpha_fi(normal_feat_imp1, 0.8)

0.8888888888888888